In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow import keras 
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, ReLU, Softmax
from sklearn.model_selection import StratifiedShuffleSplit

import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline


In [ ]:
df = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')

In [ ]:
df.shape

In [ ]:
df.head() 

In [ ]:
df.describe()

In [ ]:
val = df[df['Cover_Type']==5]


df.loc[val.index, 'Cover_Type']  = 4
print(df.iloc[val.index]['Cover_Type'] )
df.groupby(['Cover_Type']).size()


In [ ]:
df.columns

In [ ]:
numeric_values = df.columns
numeric_values

In [ ]:
df.info()

In [ ]:
len(df.Cover_Type.unique())

In [ ]:



split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df , df['Cover_Type']):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]
print(strat_train_set.shape, strat_test_set.shape)


In [ ]:

if val.index.values in strat_test_set.index.values : 
    strat_test_set.loc[val.index, 'Cover_Type']  = 5
else : 
    strat_train_set.loc[val.index, 'Cover_Type']  = 5


In [ ]:
strat_test_set.drop(labels= 'Id',axis = 1,  inplace = True)
strat_train_set.drop(labels= 'Id',axis = 1,  inplace = True)

In [ ]:
plt.figure(figsize = [20, 15]) # ploting a graph with 20  * 15 shape 
sns.heatmap(df[numeric_values[:10]].corr(), annot = True, fmt = '.4g',
           cmap = 'vlag_r', center = 0.4) #using seaborn (seaborn is a plotting library in python such as matpltlib ) to create heatmap 
plt.title('  heatmap for the first 10 numerical value')

plt.xlabel('numerical value') 
plt.ylabel('the value of this numerical value with itself')
plt.show()

In [ ]:
train_labels = strat_train_set.Cover_Type
train =strat_train_set.drop(labels='Cover_Type', axis=1 )
val_labels =strat_test_set.Cover_Type
val= strat_test_set.drop(labels='Cover_Type', axis=1 )


In [ ]:
strat_train_set.groupby(['Cover_Type']).size()


In [ ]:
strat_test_set.groupby(['Cover_Type']).size()


In [ ]:
del df 

# define standard scaler
scaler = StandardScaler()
# transform data
train = scaler.fit_transform(train)
val = scaler.fit_transform(val)
del strat_train_set, strat_test_set

In [ ]:
def onehotencode(zerosarray, data): 
    for i in range (len(data)):
        zerosarray[i][data[i]-1] = 1
    return zerosarray

In [ ]:
y_train = np.array(train_labels) # one hot encode
y_train_enc= np.zeros(len(y_train)* y_train.max()).reshape(len(y_train), y_train.max())
y_train_enc =  onehotencode(y_train_enc, y_train)



y_val = np.array(val_labels)     # one hot encode
y_val_enc = np.zeros(len(y_val)* y_val.max()).reshape(len(y_val),  y_val.max())
y_val_enc =  onehotencode(y_val_enc,  y_val)



(train.shape) ,(y_train_enc.shape), (val.shape), (y_val_enc.shape)

In [ ]:
from xgboost import XGBClassifier

params = {'colsample_bytree': 0.8547,
 'gamma': 4.3533,
 'learning_rate': 0.2502,
 'max_depth': 8,
 'n_estimators': 60,
 'subsample': 0.74}
model = XGBClassifier(**params)


In [ ]:
history =model.fit(train, y_train,
          early_stopping_rounds=200,
          eval_set=[(val,y_val)],
          verbose=True)

In [ ]:
def plot_graphs(history, string ): 
    plt.plot(history.history[string])
    plt.plot(history.history['val_' +string])
    plt.xlabel ('Epochs')
    plt.ylabel(string)
    plt.legend([string, 'val_' + string])
    plt.show()

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')
subm = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')
test.drop(labels= 'Id',axis = 1,  inplace = True)


In [ ]:
# define standard scaler
scaler = StandardScaler()
# transform data
data = scaler.fit_transform(test)
data.shape

In [ ]:
y_predict = model.predict(data)

In [ ]:
y_predicted = pd.DataFrame(y_predict)
newY = pd.concat([subm.drop(columns='Cover_Type'), y_predicted], axis = 1)
newY.columns=['Id', 'Cover_Type']
newY

In [ ]:
newY.Cover_Type.unique()

In [ ]:
newY.groupby(['Cover_Type']).size()

In [ ]:
newY.to_csv('my_submission.csv', index= False)